In [2]:
!pip install cmake

In [3]:
!pip install dlib imutils

  Running setup.py clean for dlib
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25860 sha256=6da2bcb327dfead0e8a3fb074e26e1e37cfe7adcc8e836e7d0b45a395e7206f3
  Stored in directory: c:\users\zions\appdata\local\pip\cache\wheels\86\d7\0a\4923351ed1cec5d5e24c1eaf8905567b02a0343b24aa873df2
Successfully built imutils
Failed to build dlib
    Running setup.py install for dlib: started

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\zions\anaconda3\envs\tf2\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\zions\\AppData\\Local\\Temp\\pip-install-v52iuul8\\dlib_48e2f927018a4753b786676e0bc6244e\\setup.py'"'"'; __file__='"'"'C:\\Users\\zions\\AppData\\Local\\Temp\\pip-install-v52iuul8\\dlib_48e2f927018a4753b786676e0bc6244e\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\zions\AppData\Local\Temp\pip-wheel-2_9t7zjx'
       cwd: C:\Users\zions\AppData\Local\Temp\pip-install-v52iuul8\dlib_48e2f927018a4753b786676e0bc6244e\
  Complete output (58 lines):
  running bdist_wheel
  running build
  running build_py
  package init file 'tools\python\dlib\__init__.py' not found (or not a regular file)
  running build_ext
  Building extension for Python 3.7.10 (default, Feb 26 2


    Running setup.py install for dlib: finished with status 'error'


In [4]:
# Necessary imports
import cv2
import dlib
import numpy as np
import os
import imutils

ModuleNotFoundError: No module named 'dlib'

In [23]:
## set directories
IMG_DIR_PATH = r"E:\Users\inje\교육\202012_국비지원 IT교육\SideProj.-Mask_Detection\images"
file_list = os.listdir(IMG_DIR_PATH)

#Initialize color [color_type] = (Blue, Green, Red)
color_blue = (239,207,137)
color_cyan = (255,200,0)
color_black = (0, 0, 0)
file_list[:5]

[]

### 이미지 전처리

In [ ]:
# Loading the image and resizing, converting it to grayscale
img= cv2.imread('image_path')
img = imutils.resize(img, width = 500)
gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

### dlib, OpenCV 및 Python을 사용하여 얼굴 랜드 마크 감지 및 추출

In [29]:
# Initialize dlib's face detector
detector = dlib.get_frontal_face_detector()

"""
Detecting faces in the grayscale image and creating an object - faces to store the list of bounding rectangles coordinates
The "1" in the second argument indicates that we should upsample the image 1 time.  
This will make everything bigger and allow us to detect more faces
"""

faces = detector(gray, 1)

# printing the coordinates of the bounding rectangles
print(faces)
print("Number of faces detected: ", len(faces))

"""
# Using a for loop in order to extract the specific coordinates (x1,x2,y1,y2)
for face in faces:
  x1 = face.left()
  y1 = face.top()
  x2 = face.right()
  y2 = face.bottom()
  # Drawing a rectangle around the face detected
  cv2.rectangle(img, (x1,y1), (x2,y2),(0,255,0),3)
cv2.imshow("image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
"""

NameError: name 'gray' is not defined

In [10]:
# Path of file
p = "shape_predictor_68_face_landmarks.dat"
# Initialize dlib's shape predictor
predictor = dlib.shape_predictor(p)

# Get the shape using the predictor

for face in faces:
    landmarks = predictor(gray, face)

    # for n in range(0,68):
    #     x = landmarks.part(n).x
    #     y = landmarks.part(n).y
    #     img_landmark = cv2.circle(img, (x, y), 4, (0, 0, 255), -1)

RuntimeError: Unable to open shape_predictor_68_face_landmarks.dat

In [11]:
points = []
for i in range(1, 16):
    point = [landmarks.part(i).x, landmarks.part(i).y]
    points.append(point)
    # print(points)

    # Coordinates for the additional 3 points for wide, high coverage mask - in sequence
    mask_a = [((landmarks.part(42).x), (landmarks.part(15).y)),
              ((landmarks.part(27).x), (landmarks.part(27).y)),
              ((landmarks.part(39).x), (landmarks.part(1).y))]

    # Coordinates for the additional point for wide, medium coverage mask - in sequence
    mask_c = [((landmarks.part(29).x), (landmarks.part(29).y))]

    # Coordinates for the additional 5 points for wide, low coverage mask (lower nose points) - in sequence
    mask_e = [((landmarks.part(35).x), (landmarks.part(35).y)),
              ((landmarks.part(34).x), (landmarks.part(34).y)),
              ((landmarks.part(33).x), (landmarks.part(33).y)),
              ((landmarks.part(32).x), (landmarks.part(32).y)),
              ((landmarks.part(31).x), (landmarks.part(31).y))]

    fmask_a = points + mask_a
    fmask_c = points + mask_c
    fmask_e = points + mask_e

    # mask_type = {1: fmask_a, 2: fmask_c, 3: fmask_e}
    # mask_type[choice2]

    # Using Python OpenCV – cv2.polylines() method to draw mask outline for [mask_type]:
    # fmask_a = wide, high coverage mask,
    # fmask_c = wide, medium coverage mask,
    # fmask_e  = wide, low coverage mask

    fmask_a = np.array(fmask_a, dtype=np.int32)
    fmask_c = np.array(fmask_c, dtype=np.int32)
    fmask_e = np.array(fmask_e, dtype=np.int32)

    mask_type = {1: fmask_a, 2: fmask_c, 3: fmask_e}
    mask_type[choice2]

    # change parameter [mask_type] and color_type for various combination
    img2 = cv2.polylines(img, [mask_type[choice2]], True, choice1, thickness=2, lineType=cv2.LINE_8)

    # Using Python OpenCV – cv2.fillPoly() method to fill mask
    # change parameter [mask_type] and color_type for various combination
    img3 = cv2.fillPoly(img2, [mask_type[choice2]], choice1, lineType=cv2.LINE_AA)

    # cv2.imshow("image with mask outline", img2)
    cv2.imshow("image with mask", img3)

#Save the output file for testing
outputNameofImage = "output/imagetest.jpg"
print("Saving output image to", outputNameofImage)
cv2.imwrite(outputNameofImage, img3)

IndentationError: unexpected indent (<ipython-input-11-48165b571134>, line 2)

In [12]:
points = []
for i in range(1, 16):
    point = [landmarks.part(i).x, landmarks.part(i).y]
    points.append(point)
    # print(points)

    # Ellipse parameters for high, round coverage mask
    top_ellipse = landmarks.part(27).y + (landmarks.part(28).y - landmarks.part(27).y) / 2
    centre_x = landmarks.part(28).x
    centre_y = landmarks.part(8).y - ((landmarks.part(8).y - (top_ellipse)) / 2)
    # (height of ellipse)
    axis_major = (landmarks.part(8).y - top_ellipse) / 2
    # (width of ellipse)
    axis_minor = ((landmarks.part(13).x - landmarks.part(3).x) * 0.8) / 2

    centre_x = int(round(centre_x))
    centre_y = int(round(centre_y))
    axis_major = int(round(axis_major))
    axis_minor = int(round(axis_minor))

    centre = (centre_x, centre_y)
    axes = (axis_major, axis_minor)

    # Using Python OpenCV – cv2.ellipse() method to draw mask outline for mask
    # change last parameter - line thickness and color_type for various combination
    img_2 = cv2.ellipse(img, centre, axes, 0, 0, 360, color_type, thickness=2)



    # Using Python OpenCV – cv2.ellipse() method to draw mask outline for mask
    # change last parameter - line thickness to negative for fill and color_type for various combination
    img_3 = cv2.ellipse(img, centre, axes, 0, 0, 360, color_type, thickness=-1)

# cv2.imshow("image with mask outline", img_2)
cv2.imshow("image with mask", img_3)

IndentationError: unexpected indent (<ipython-input-12-b3c751728faa>, line 2)

In [13]:
# Use input () function to capture from user requirements for mask type and mask colour
choice1 = input("Please select the choice of mask color\nEnter 1 for blue\nEnter 2 for black:\n")
choice1 = int(choice1)

if choice1 == 1:
    choice1 = color_blue
    print('You selected mask color = blue')
elif choice1 == 2:
    choice1 = color_black
    print('You selected mask color = black')
else:
    print("invalid selection, please select again.")
    input("Please select the choice of mask color\nEnter 1 for blue\nEnter 2 for black :\n")


choice2 = input("Please enter choice of mask type coverage \nEnter 1 for high \nEnter 2 for medium \nEnter 3 for low :\n")
choice2 = int(choice2)

if choice2 == 1:
    choice2 = fmask_a
    print(f'You chosen wide, high coverage mask')
elif choice2 == 2:
    choice2 = fmask_c
    print(f'You chosen wide, medium coverage mask')
elif choice2 == 3:
    choice2 = fmask_e
    print(f'You chosen wide, low coverage mask')
else:
    print("invalid selection, please select again.")
    input("Please enter choice of mask type coverage \nEnter 1 for high \nEnter 2 for medium \nEnter 3 for low :\n")

# print(choice2)

KeyboardInterrupt: Interrupted by user

Please select the choice of mask color
Enter 1 for blue
Enter 2 for black:
2


https://www.kaggle.com/niharika41298/withwithout-mask

# 마스크씌우기 파이프라인

In [21]:
# Necessary imports
import cv2
import dlib
import numpy as np
import os
import imutils

In [22]:
## set directories
IMG_DIR_PATH = r"C:\Users\inje.jeong\Desktop\images"
os.chdir(IMG_DIR_PATH)
file_list = os.listdir(IMG_DIR_PATH)

#Initialize color [color_type] = (Blue, Green, Red)
color_blue = (239,207,137)
color_cyan = (255,200,0)
color_black = (0, 0, 0)
color_white = (255,255,255)
file_list[:5]

['0.jpg', '1.jpg', '10.jpg', '100.jpg', '101.jpg']

In [23]:
# # IMG_PATH = os.path.join(IMG_DIR_PATH, file_list[0])
# img = cv2.imread(file_list[0])
# # img
# cv2.imshow('img',img)
# cv2.waitKey(0) # 키보드 입력을 기다린다. (0: 입력될 떄까지 기다린다. 양수=ms(밀리초))
# cv2.destroyAllWindows()

In [24]:
# IMG_PATH = os.path.join(IMG_DIR_PATH, file_list[0])
# print(IMG_PATH)
# os.chdir(IMG_DIR_PATH)

In [25]:
# img = cv2.imread(IMG_PATH)
# # img
# cv2.imshow('img',img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [26]:
os.getcwd()

'C:\\Users\\inje.jeong\\Desktop\\images'

In [33]:
# Loading the image and resizing, converting it to grayscale
choice1 = color_white # 마스크 색깔
choice2 = 2 # 마스크 모양 0 ~ 2

# 이미지 있는 디렉토리로 설정
IMG_DIR_PATH = r"C:\Users\inje.jeong\Desktop\images"
# 주피터노트북 파일 있는 디렉토리로 설정
BASE_PATH = r"E:\Users\inje\교육\202012_국비지원 IT교육\SideProj.-Mask_Detection"
os.chdir(IMG_DIR_PATH)
if not os.path.isdir('output'):
    os.mkdir('output')
OUTPUT_DIR_PATH = os.path.join(IMG_DIR_PATH, 'output')
os.chdir(BASE_PATH)

for idx, img_path in enumerate(file_list):
    IMG_PATH = os.path.join(IMG_DIR_PATH, img_path)
    img = cv2.imread(IMG_PATH)
    img = imutils.resize(img, width = 500)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Initialize dlib's face detector
    detector = dlib.get_frontal_face_detector()
    
    faces = detector(gray, 1)
    
    # Initialize dlib's shape predictor
    # Get the shape using the predictor
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    for face in faces:
        landmarks = predictor(gray, face)

        points = []
        for i in range(1, 16):
            point = [landmarks.part(i).x, landmarks.part(i).y]
            points.append(point)

        # Coordinates for the additional 3 points for wide, high coverage mask - in sequence
        mask_a = [((landmarks.part(42).x), (landmarks.part(15).y)),
                  ((landmarks.part(27).x), (landmarks.part(27).y)),
                  ((landmarks.part(39).x), (landmarks.part(1).y))]

        # Coordinates for the additional point for wide, medium coverage mask - in sequence
        mask_c = [((landmarks.part(29).x), (landmarks.part(29).y))]

        # Coordinates for the additional 5 points for wide, low coverage mask (lower nose points) - in sequence
        mask_e = [((landmarks.part(35).x), (landmarks.part(35).y)),
                  ((landmarks.part(34).x), (landmarks.part(34).y)),
                  ((landmarks.part(33).x), (landmarks.part(33).y)),
                  ((landmarks.part(32).x), (landmarks.part(32).y)),
                  ((landmarks.part(31).x), (landmarks.part(31).y))]

        fmask_a = points + mask_a
        fmask_c = points + mask_c
        fmask_e = points + mask_e

        # mask_type = {1: fmask_a, 2: fmask_c, 3: fmask_e}
        # mask_type[choice2]

        # Using Python OpenCV – cv2.polylines() method to draw mask outline for [mask_type]:
        # fmask_a = wide, high coverage mask,
        # fmask_c = wide, medium coverage mask,
        # fmask_e  = wide, low coverage mask

        fmask_a = np.array(fmask_a, dtype=np.int32)
        fmask_c = np.array(fmask_c, dtype=np.int32)
        fmask_e = np.array(fmask_e, dtype=np.int32)

        mask_type = {1: fmask_a, 2: fmask_c, 3: fmask_e}
        # mask_type[choice2]

        # change parameter [mask_type] and color_type for various combination
        img2 = cv2.polylines(img, [mask_type[choice2]], True, choice1,
                             thickness=2, lineType=cv2.LINE_8)

        # Using Python OpenCV – cv2.fillPoly() method to fill mask
        # change parameter [mask_type] and color_type for various combination
        img3 = cv2.fillPoly(img2, [mask_type[choice2]], choice1, lineType=cv2.LINE_AA)

        # cv2.imshow("image with mask outline", img2)
        # cv2.imshow("image with mask", img3)

#     cv2.imshow("image with mask outline", img_2)
#     cv2.imshow(f"image with mask_{idx}", img3)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

    #Save the output file for testing
    outputNameofImage = f"output_{idx}.jpg"
    print("Saving output image to", outputNameofImage)
    OUTPUT_PATH = os.path.join(OUTPUT_DIR_PATH, outputNameofImage)
    cv2.imwrite(OUTPUT_PATH, img3)

Saving output image to output_0.jpg
Saving output image to output_1.jpg
Saving output image to output_2.jpg
Saving output image to output_3.jpg
Saving output image to output_4.jpg
Saving output image to output_5.jpg
Saving output image to output_6.jpg
Saving output image to output_7.jpg
Saving output image to output_8.jpg
Saving output image to output_9.jpg
Saving output image to output_10.jpg
Saving output image to output_11.jpg
Saving output image to output_12.jpg
Saving output image to output_13.jpg
Saving output image to output_14.jpg
Saving output image to output_15.jpg
Saving output image to output_16.jpg
Saving output image to output_17.jpg
Saving output image to output_18.jpg
Saving output image to output_19.jpg
Saving output image to output_20.jpg
Saving output image to output_21.jpg
Saving output image to output_22.jpg
Saving output image to output_23.jpg
Saving output image to output_24.jpg
Saving output image to output_25.jpg
Saving output image to output_26.jpg
Saving outp

Saving output image to output_219.jpg
Saving output image to output_220.jpg
Saving output image to output_221.jpg
Saving output image to output_222.jpg
Saving output image to output_223.jpg
Saving output image to output_224.jpg
Saving output image to output_225.jpg
Saving output image to output_226.jpg
Saving output image to output_227.jpg
Saving output image to output_228.jpg
Saving output image to output_229.jpg
Saving output image to output_230.jpg
Saving output image to output_231.jpg
Saving output image to output_232.jpg
Saving output image to output_233.jpg
Saving output image to output_234.jpg
Saving output image to output_235.jpg
Saving output image to output_236.jpg
Saving output image to output_237.jpg
Saving output image to output_238.jpg
Saving output image to output_239.jpg
Saving output image to output_240.jpg
Saving output image to output_241.jpg
Saving output image to output_242.jpg
Saving output image to output_243.jpg
Saving output image to output_244.jpg
Saving outpu

### 타원형 마스킹 파이프라인

In [ ]:
# # Loading the image and resizing, converting it to grayscale
# choice1 = color_white # 마스크 색깔
# choice2 = 2 # 마스크 모양 0 ~ 2

# for idx, img_path in enumerate(file_list):
#     IMG_PATH = os.path.join(IMG_DIR_PATH, img_path)
#     img = cv2.imread(IMG_PATH)
#     img = imutils.resize(img, width = 500)
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
#     # Initialize dlib's face detector
#     detector = dlib.get_frontal_face_detector()
    
#     faces = detector(gray, 1)
    
#     # Initialize dlib's shape predictor
#     # Get the shape using the predictor
#     predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

#     for face in faces:
#         landmarks = predictor(gray, face)

#         points = []
#         for i in range(1, 16):
#             point = [landmarks.part(i).x, landmarks.part(i).y]
#             points.append(point)

#         # Ellipse parameters for high, round coverage mask
#         top_ellipse = landmarks.part(27).y + (landmarks.part(28).y - landmarks.part(27).y) / 2
#         centre_x = landmarks.part(28).x
#         centre_y = landmarks.part(8).y - ((landmarks.part(8).y - (top_ellipse)) / 2)
#         # (height of ellipse)
#         axis_major = (landmarks.part(8).y - top_ellipse) / 2
#         # (width of ellipse)
#         axis_minor = ((landmarks.part(13).x - landmarks.part(3).x) * 0.8) / 2

#         centre_x = int(round(centre_x))
#         centre_y = int(round(centre_y))
#         axis_major = int(round(axis_major))
#         axis_minor = int(round(axis_minor))

#         centre = (centre_x, centre_y)
#         axes = (axis_major, axis_minor)

#         # Using Python OpenCV – cv2.ellipse() method to draw mask outline for mask
#         # change last parameter - line thickness and color_type for various combination
#         img4 = cv2.ellipse(img, centre, axes, 0, 0, 360, choice1, thickness=2)

#         # Using Python OpenCV – cv2.ellipse() method to draw mask outline for mask
#         # change last parameter - line thickness to negative for fill and color_type for various combination
#         img5 = cv2.ellipse(img, centre, axes, 0, 0, 360, choice1, thickness=-1)

#         # cv2.imshow("image with mask outline", img_2)
#         cv2.imshow(f"image with mask_{idx}", img3)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()

#         #Save the output file for testing
#         outputNameofImage = f"output/imagetest_{idx}.jpg"
#         print("Saving output image to", outputNameofImage)
#         cv2.imwrite(outputNameofImage, img5)